## Weekly Project Status

:::: {.columns}

::: {.column width="33%"}
<div class="status-card">

**Last Week**

- Assay validation complete
- 500 compounds analyzed
- MSD panel optimized
- SAR presentation delivered
- Z' factor achieved 0.82 for assay X
- Cross-functional alignment meeting
- Automated data pipeline deployed

</div>
:::

::: {.column width="33%"}
<div class="status-card">

**This Week**

- Ex vivo PBMC assay development
- Hamilton automation implementation
- Z' performance analysis
- Preparation for translational strategy meeting
- Troubleshooting CV variability for assay Y
- Stakeholder updates scheduled
- DOE optimization for hit validation

</div>
:::

::: {.column width="33%"}
<div class="status-card">

**Next Week**

- Launch HTS (10K compounds)
- TME co-culture validation
- QC dashboard review
- OpenTrons demo session
- Scale to 384-well format for assay Z
- Integrate MSD biomarker panel
- Present findings to leadership team

</div>
:::

::::

## Assay Quality Control Dashboard

:::: {.columns}

::: {.column width="20%"}

<div style="padding: 0.6em; background: rgba(0, 48, 135, 0.05); border-radius: 6px; margin-bottom: 0.5em;">

**Assay Robustness**

- **Z' Factor:** <span style="color: #76bc21; font-weight: bold;">0.82</span> ✓
- **CV%:** <span style="color: #76bc21; font-weight: bold;">8.2%</span> ✓
- **Data Quality:** <span style="color: #76bc21; font-weight: bold;">99.2%</span>

</div>

<div style="padding: 0.6em; background: rgba(0, 48, 135, 0.05); border-radius: 6px;">

**Assay Performance**

- **Signal-to-Background:** <span style="color: #00a3b5; font-weight: bold;">95%</span>
- **S/B Ratio:** <span style="color: #00a3b5; font-weight: bold;">12.5</span>
- **Throughput:** <span style="color: #00a3b5; font-weight: bold;">3,840/week</span>

</div>

:::

::: {.column width="80%"}

In [5]:
import pandas as pd
import numpy as np
import altair as alt

# Create sample data with ELN links
qc_data = pd.DataFrame({
    'date': pd.to_datetime([
        '2024-01-15', '2024-01-28', '2024-02-12', '2024-02-25',
        '2024-03-10', '2024-04-05', '2024-05-08', '2024-06-03',
        '2024-06-20', '2024-07-02', '2024-07-18', '2024-08-05'
    ]),
    'z_prime': [0.35, 0.42, 0.58, 0.65, 0.68, 0.74, 0.78, 0.80,
                0.85, 0.77, 0.82, 0.88],
    'std_dev': [0.12, 0.10, 0.08, 0.07, 0.06, 0.05, 0.055, 0.045,
                0.040, 0.048, 0.042, 0.035],
    'assay_id': ['ASY-001', 'ASY-002', 'ASY-003', 'ASY-004',
                 'ASY-005', 'ASY-006', 'ASY-007', 'ASY-008',
                 'ASY-009', 'ASY-010', 'ASY-011', 'ASY-012'],
    'test_condition': ['Control A', 'Control A', 'Control A', 'Control B',
                       'Control B', 'Control B', 'Modified pH', 'Modified pH',
                       'Modified pH', 'High Serum', 'High Serum', 'High Serum'],
    'eln_url': [f'https://demo.elabftw.net/experiments.php?mode=view&id={i}' for i in range(1, 13)]
})

# Format date for display
qc_data['date_label'] = qc_data['date'].dt.strftime('%b %d')

# Create color scale based on Z' factor value
qc_data['status'] = qc_data['z_prime'].apply(
    lambda x: 'Excellent' if x >= 0.7 else ('Good' if x >= 0.5 else 'Below Target')
)

# Create the interactive Altair chart
base = alt.Chart(qc_data).encode(
    x=alt.X('yearmonth(date):T', 
            scale=alt.Scale(padding=30),
            axis=alt.Axis(title='Assay Date (2026)', 
                         format='%b',
                         titleFontSize=18,
                         titleFontWeight='bold',
                         titleColor='#2c2c2c',
                         labelFontSize=16,
                         labelColor='#2c2c2c',
                         grid=False)),
    y=alt.Y('z_prime:Q', 
            scale=alt.Scale(domain=[0, 1]),
            axis=alt.Axis(title="Z' Factor",
                         titleFontSize=18,
                         titleFontWeight='bold',
                         titleColor='#2c2c2c',
                         labelFontSize=16,
                         labelColor='#2c2c2c',
                         grid=False)),
    color=alt.Color('test_condition:N',
                   scale=alt.Scale(domain=['Control A', 'Control B', 'Modified pH', 'High Serum'],
                                 range=['#003087', '#00a3b5', '#76bc21', '#ff6f31']),
                   legend=alt.Legend(
                       title='Testing Condition',
                       titleFontSize=14,
                       titleFontWeight='bold',
                       labelFontSize=12,
                       orient='right',
                       direction='vertical'
                   )),
    size=alt.Size('std_dev:Q',
                 scale=alt.Scale(domain=[0.03, 0.12], range=[100, 400]),
                 legend=alt.Legend(
                     title='Std Dev (σ)',
                     titleFontSize=14,
                     titleFontWeight='bold',
                     labelFontSize=12,
                     orient='right',
                     direction='vertical',
                     format='.3f'
                 )),
    href='eln_url:N',
    tooltip=[
        alt.Tooltip('date_label:N', title='Date'),
        alt.Tooltip('z_prime:Q', title="Z' Factor", format='.2f'),
        alt.Tooltip('std_dev:Q', title='Std Dev', format='.3f'),
        alt.Tooltip('test_condition:N', title='Test Condition'),
        alt.Tooltip('assay_id:N', title='Assay ID'),
        alt.Tooltip('status:N', title='Status')
    ]
)

# Points layer with interactivity
points = base.mark_circle(opacity=0.8, cursor='pointer')

# Target line
target_line = alt.Chart(pd.DataFrame({'y': [0.5]})).mark_rule(
    strokeDash=[5, 5],
    color='#ff6f31',
    size=3
).encode(y='y:Q')

# Target label
target_text = alt.Chart(pd.DataFrame({
    'x': [qc_data['date'].max()],
    'y': [0.52],
    'text': ['Target: 0.5']
})).mark_text(
    align='right',
    color='#ff6f31',
    fontSize=15
).encode(
    x='yearmonth(x):T',
    y='y:Q',
    text='text:N'
)

# Combine layers
chart = (points + target_line + target_text).properties(
    width=890,
    height=525,
    title=alt.TitleParams(
        text="Z' Factor Performance Over Time For Generic Fictional Assay",
        fontSize=24,
        fontWeight='bold',
        color='#003087',
        anchor='middle'
    )
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
).configure_legend(
    strokeColor='gray',
    fillColor='#f9f9f9',
    padding=10,
    cornerRadius=5
)

chart

alt.LayerChart(...)

:::

::::

## Bio Toolbox

<div class="merck-blue-box">

**Interactive Data Analysis & Visualization Tools**

</div>

:::: {.columns}

::: {.column width="33%"}
<div class="feature-card">

**[Plate Mapper](https://www.plate.mcvcllmhgb.com/)**

Interactive microtiter-plate visualization for general purpose experiment planning and on the fly automation friendly dose planning.

</div>
:::

::: {.column width="33%"}
<div class="feature-card">

**[Dose-Response Curves](https://doseresponse.mcvcllmhgb.com/)**

Quick and dirty tool for curve fitting and teasing out optimal fit equation from common set against EIC and BIC.

</div>
:::

::: {.column width="33%"}
<div class="feature-card">

**Additional Tools**

Extension on Plate Mapper for generating plate plans from compound list. LLM powered bench protocol editing and aesthetic design suite (mcvcllmhgb.com). And much more...

</div>
:::

::::

<div class="highlight-box" style="margin-top: 0.5em; text-align: center; font-size: 0.75em; padding: 0.5em;">

**Open source applications** for internal use and cross-functional collaboration

</div>